## **Supervised Fine-tuning**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch

torch.cuda.is_available()


True

testing the tokenizer.

In [ ]:
text = "Hello, Saif Rathod this side!"
tokens = tokenizer(text)
print(tokens)
#15496, 11, 10318, 361, 26494, 375, 428, 1735, 0

{'input_ids': [15496, 11, 10318, 361, 26494, 375, 428, 1735, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
tokenizer.decode(tokens['input_ids'])

'Hello, Saif Rathod this side!'

In [ ]:
%pip install datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/sst2")

In [ ]:
ds

DatasetDict({
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
})

In [ ]:
ds_train, ds_val = ds['train'], ds['validation']

In [ ]:
ds_train[6]

{'idx': 6,
 'sentence': 'demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
 'label': 1}

In [ ]:
ds_train[:10]

{'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'sentence': ['hide new secretions from the parental units ',
  'contains no wit , only labored gags ',
  'that loves its characters and communicates something rather beautiful about human nature ',
  'remains utterly satisfied to remain the same throughout ',
  'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
  "that 's far too tragic to merit such superficial treatment ",
  'demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
  'of saucy ',
  "a depressed fifteen-year-old 's suicidal poetry ",
  "are more deeply thought through than in most ` right-thinking ' films "],
 'label': [0, 0, 1, 0, 0, 0, 1, 1, 0, 1]}

tokenize the dataset

In [ ]:
def tokenize(batch):
  return tokenizer(batch['sentence'])

map_kwargs = {
    'batched' : True,
    'batch_size' : 512,
    'remove_columns' : ['idx', 'sentence', 'label']
}

tokenized_ds_train = ds_train.map(tokenize, **map_kwargs)
tokenized_ds_val = ds_val.map(tokenize, **map_kwargs)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds_train[0]

{'input_ids': [24717, 649, 3200, 507, 422, 262, 21694, 4991, 220],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenized_ds_train[5:10]

{'input_ids': [[5562,
   705,
   82,
   1290,
   1165,
   15444,
   284,
   17004,
   884,
   31194,
   3513,
   220],
  [26567,
   2536,
   689,
   326,
   262,
   3437,
   286,
   884,
   289,
   31777,
   2512,
   30181,
   355,
   29408,
   1830,
   460,
   991,
   1210,
   503,
   257,
   1402,
   837,
   2614,
   2646,
   351,
   281,
   7016,
   3355,
   404,
   764,
   220],
  [1659, 473, 84, 948, 220],
  [64, 19095, 17280, 12, 1941, 12, 727, 705, 82, 26781, 19518, 220],
  [533,
   517,
   7744,
   1807,
   832,
   621,
   287,
   749,
   4600,
   826,
   12,
   28973,
   705,
   7328,
   220]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
for i, seq in enumerate(tokenized_ds_train[5:10]['input_ids']):
  print(f"Tokenized: {tokenizer.decode(seq)}")

Tokenized: that 's far too tragic to merit such superficial treatment 
Tokenized: demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . 
Tokenized: of saucy 
Tokenized: a depressed fifteen-year-old 's suicidal poetry 
Tokenized: are more deeply thought through than in most ` right-thinking ' films 


In [ ]:
len(tokenized_ds_train), len(tokenized_ds_val)

(67349, 872)

In [ ]:
tokenized_ds_train = tokenized_ds_train.filter(lambda x: len(x['input_ids']) > 5)

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds_val = tokenized_ds_val.filter(lambda x: len(x['input_ids']) > 5)

Filter:   0%|          | 0/872 [00:00<?, ? examples/s]

In [ ]:
len(tokenized_ds_train), len(tokenized_ds_val)

(49401, 867)

## Preparing a dataloader

In [ ]:
import numpy as np
print(np.__version__)

2.0.2


In [ ]:
!pip uninstall -y numpy && pip install numpy==1.26.4


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
tokenized_ds_train.set_format(type='torch')
tokenized_ds_val.set_format(type='torch')

In [ ]:
tokenized_ds_train[0]


{'input_ids': tensor([24717,   649,  3200,   507,   422,   262, 21694,  4991,   220]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [ ]:
tokenized_ds_train[:5]

{'input_ids': [tensor([24717,   649,  3200,   507,   422,   262, 21694,  4991,   220]),
  tensor([ 3642,  1299,   645, 20868,   837,   691,  2248,  1850,   308,  3775,
            220]),
  tensor([ 5562, 10408,   663,  3435,   290, 48556,  1223,  2138,  4950,   546,
           1692,  3450,   220]),
  tensor([ 2787,  1299, 15950, 11378,   284,  3520,   262,   976,  3690,   220]),
  tensor([  261,   262,  5290, 15827,    12,  1659,    12,  1169,    12,  1008,
           9310, 35478, 20954,   262, 28303,   714, 47478,   469,   510,   220])],
 'attention_mask': [tensor([1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]}

### Padding


In [ ]:
print(tokenizer.pad_token)

None


In [ ]:
print(tokenizer.eos_token)

<|endoftext|>


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

dataloader_params = {
    'batch_size' : 32,
    'collate_fn' : data_collator
}

train_dataloader = DataLoader(tokenized_ds_train, **dataloader_params)
val_dataloader = DataLoader(tokenized_ds_val, **dataloader_params)

In [ ]:
len(train_dataloader)

1544

In [ ]:
1544 * 32

49408

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())

KeysView({'input_ids': tensor([[24717,   649,  3200,  ..., 50256, 50256, 50256],
        [ 3642,  1299,   645,  ..., 50256, 50256, 50256],
        [ 5562, 10408,   663,  ..., 50256, 50256, 50256],
        ...,
        [  672, 35260,   284,  ..., 50256, 50256, 50256],
        [ 1169,  2104,   966,  ..., 50256, 50256, 50256],
        [  292,   484,  1282,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[24717,   649,  3200,  ...,  -100,  -100,  -100],
        [ 3642,  1299,   645,  ...,  -100,  -100,  -100],
        [ 5562, 10408,   663,  ...,  -100,  -100,  -100],
        ...,
        [  672, 35260,   284,  ...,  -100,  -100,  -100],
        [ 1169,  2104,   966,  ...,  -100,  -100,  -100],
        [  292,   484,  1282,  ...,  -100,  -100,  -100]])})


In [ ]:
batch['input_ids'].shape  #batch_size, seq_length

torch.Size([32, 35])

In [ ]:
batch['input_ids'][0]

tensor([24717,   649,  3200,   507,   422,   262, 21694,  4991,   220, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256])

In [ ]:
batch['labels'][0]

tensor([24717,   649,  3200,   507,   422,   262, 21694,  4991,   220,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100])

In [ ]:
batch['attention_mask'][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## SFT

In [ ]:
import torch
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 1

training loop

In [ ]:
def validate(epoch):
    model.eval()
    total_loss = 0.0
    for i, batch in enumerate(val_dataloader):
        # iteration = epoch * len(val_dataloader) + i
        batch = batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
            loss = outputs.loss # Uses transformers.loss.loss_utils.ForCausalLMLoss for loss calculation
            total_loss += loss.item()
    print(f'val_loss at {epoch} epoch:', total_loss / len(val_dataloader))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
validate(0)
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_dataloader):
        batch = batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        print(f'Loss: {loss.item()}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    validate(epoch+1)

val_loss at 0 epoch: 3.934447339602879
Loss: 3.4918212890625
Loss: 3.730374813079834
Loss: 3.4608428478240967
Loss: 3.70367169380188
Loss: 3.8544561862945557
Loss: 3.8919858932495117
Loss: 3.6999013423919678
Loss: 3.6266391277313232
Loss: 3.6039493083953857
Loss: 3.6997900009155273
Loss: 3.7023937702178955
Loss: 3.4633944034576416
Loss: 3.813965320587158
Loss: 3.902867317199707
Loss: 3.448918104171753
Loss: 3.4941587448120117
Loss: 3.593003511428833
Loss: 3.7093873023986816
Loss: 3.662118673324585
Loss: 3.6940207481384277
Loss: 3.622570753097534
Loss: 3.442495584487915
Loss: 3.6286044120788574
Loss: 3.6547813415527344
Loss: 3.6707489490509033
Loss: 3.48372220993042
Loss: 3.656770944595337
Loss: 3.5214474201202393
Loss: 3.6530158519744873
Loss: 3.6252589225769043
Loss: 3.6937968730926514
Loss: 3.6192851066589355
Loss: 3.4953272342681885
Loss: 3.698779821395874
Loss: 3.7242467403411865
Loss: 3.5429930686950684
Loss: 3.618696451187134
Loss: 3.5880227088928223
Loss: 3.658062219619751
Loss:

In [ ]:
print(loss.item())

3.1061930656433105


In [ ]:
model.save_pretrained('./sft_model')

In [ ]:
model.from_pretrained('./sft_model')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)